# Von XML zur Tabelle
In diesem Tutorial sehen wir uns an, wie wir XML-Dokumente auslesen können, und diese in eine Tabelle transformieren, mit der wir dann praktisch weiterarbeiten können. Welche Elemente wir wie ins XML übernehmen wollen, ist abhängig von der Fragestellung. In diesem Beispiel werden wir die annotierten Orte auslesen und in einer Tabelle verzeichnen. Diese Tabelle dient dann auch als Grundlage für die Kartenvisualisierung.

In [ ]:
# import der relevanten Bibliotheken
from lxml import etree as et
import glob
import re

Ganz spezifisch wollen wir folgende Informationen in der Tabelle verzeichnen: 

Metadaten zum Brief
- Brief ID
- Sender:in
- Sender:in ID
- Empfänger:in
- Empfänger:in ID
- Datierung

Wir nehmen hier - etwas naiv - eine:n einzelne:n Sender:in / Empfänger:in an.

Informationen zum gefundenen Ort:
- Name des Orts
- Ort ID

Ein einfacher Weg, eine Tabelle zu bauen, ist die Informationen zuerst in einer Liste zu sammeln und die Listen dann in eine Tabelle umzuwandeln.

In [ ]:
rows = []

for letter in glob.glob("docs/*.xml"):
    tree = et.parse(letter)
    root = tree.getroot()

    letter_id = root.get("{http://www.w3.org/XML/1998/namespace}id")
    sender = root.find("./tei:teiHeader/tei:profileDesc/tei:correspDesc/tei:correspAction[@type='sent']/tei:persName", namespaces={"tei":"http://www.tei-c.org/ns/1.0"})
    sender_name = sender.text
    sender_id = sender.get("key")
    receiver = root.find("./tei:teiHeader/tei:profileDesc/tei:correspDesc/tei:correspAction[@type='received']/tei:persName", namespaces={"tei":"http://www.tei-c.org/ns/1.0"})
    receiver_name = receiver.text
    receiver_id = receiver.get("key")
    letter_date = root.find("./tei:teiHeader/tei:profileDesc/tei:correspDesc/tei:correspAction[@type='sent']/tei:date", namespaces={"tei":"http://www.tei-c.org/ns/1.0"}).get("when")

    letter_metadata = [letter_id, sender_name, sender_id, receiver_name, receiver_id, letter_date]

    places = root.xpath("./tei:text/tei:body/tei:div[@type='content']//tei:placeName", namespaces={"tei":"http://www.tei-c.org/ns/1.0"})

    for place in places:
        row = []  # in dieser Liste sammeln wir alle Infos zum jeweiligen Ort

        row.extend(letter_metadata)

        # etwas viel Code, aber räumt uns schön die Ortsnennung auf, auch bei Zeilenumbrüchen etc.
        place_name = et.tostring(place, method="text", encoding="utf8", with_tail=False).decode("utf8")
        place_name = re.sub(r"\s*\n\s*", " ", place_name)
        row.append(place_name)

        place_id = place.get("key")
        row.append(place_id)

        rows.append(row)

print(rows[:3])

Nun haben wir eine Liste mit einer Zeile pro gefundenem Ort, diese lässt sich nun sehr einfach in eine Tabelle umwandeln. Am besten ist der Umgang mit Tabellen im externen Package [*pandas*](https://pandas.pydata.org/) zu organisieren.

Das Pandas-Package enthält eine grosse Auswahl von Operationen von tabellarischen Datensätzen, auf die aber hier erstmal nicht weiter eingegangen wird.

In [ ]:
# installieren von pandas
%pip install -U pandas

In [ ]:
import pandas as pd

columns = ['letter_id', 'sender_name', 'sender_key', 'receiver_name', 'receiver_key', 'date', 'place_name', 'place_key']

df = pd.DataFrame(rows, columns=columns)

print(df.head)

In [ ]:
# speichern der Tabelle als CSV
df.to_csv('places.csv', index=False)